# Echaffaudage d'une application web en Python

## Introduction

Pour créer une application web, il faut profiter d’un serveur HTTP. 
Pour cette raison, PHP s’est très tôt imposé comme le langage de référence du web, 
car il est naturellement coupléavec Apache ou Nginx.

Pour les autres langages, c’est différent.
Il faut passer par l’interface CGI (Common Gateway Interface) du serveur, ce qui n’est pas toujours pratique.
La seconde solution consiste à se servir des ressources asynchrones des langages eux-mêmes (ce qui est plus compliqué en PHP, mais est devenu faisable).
C’est ce pourquoi NodeJS est conçu.
Pour **Python**, il serait fastidieux de faire cela manuellement avec le module **asyncio**.
On aura donc recours à des bibliothèques tierces qui serviront de « _framework_ » (plate-forme) facilitant la mise en œuvre.

Globalement, il existe deux outils majeurs en Python :

1. **Django**, qui est un framwork lourd, destiné à des applications professionnelles, équivalent à **Symfony** en PHP ; les deux s'inspirent du monde **Java**.
2. **Flask**, qui ets un framework léger, très simple à mettre en œuvre, mais qui permet de créer des applications complètes ; c'est lui ue nous utiliserons.

## Installation

Pour pouvoir créer notre application, nouaurons également besoin de :

- **Jinja**, qui est un **moteur de rendu** HTML, qui se chargera de la mise en forme des pages à afficher ;
- **SQLAlchemy**, qui exécutera les requêtes sur la base de données
- **Werkzeug**, qui rendra l'application plus puissante en utilisant le protocole WSGI

Pour tout cela :

In [ ]:
%%bash

pip install flask
pip install jinja2
pip install sqlalchemy

## Flask

### Lancement du serveur HTTP

Pour lancer le serveur, il nous suffit de créer un script Python comme le suivant :

In [ ]:
# import de l'objet Flask, qui fait office de serveur
from flask import Flask

# initialistion du serveur
# __name__ est le nom  du fichier
app = Flask(__name__)

# définition d'une route racine grâce à un décorateur
# la route est associé à un contrôleur, dont le rôle sera de traiter la requête de l'utilisateur
@app.route('/')
def home():
    return "Ceci est la page d'accueil"

Une fois le script sauvegardé, nous pouvons lancer l'pplication (qui contient son propre serveur HTTP) avec la commande :

In [ ]:
flask run

Cette commande suffit si vous avez eu la bonne idée de nommer votre fichier `app.py`. Sinon, la syntaxe sera un peu plus verbeuse :

In [ ]:
# Si le fichier s'appelle `index.py`
flask run --app index.py

Par défaut, le serveur écute sur le port 5000. Vous pouvez donc aller dans votre navigateur et interroger l’URL : `localhost:5000` pour afficher le message.

### Familles de routes

Nous pouvons maintenant ajouter autant de routes que nous voulons.
Toutefois, nous avons besoin d'éléments variables dans les URL.
Admettons que nouvousions afficher les informations d'un produit, il faudra passer son `id` au serveur.
Il faut donc une syntaxe “générique” pour une famille de routes, du type :
```
/product/15
/product/124
... c'est-à-dire :
/product/{id}
```
Dans Flask, les parties variables sont entourées par des chevrons :

In [ ]:
# Une variable est introduite dans la route
@app.route('/product/<id>')
# Cette variable est également introduite comme paramètre du contrôleur
def home(id):
    # Pour simplifier l'insertion de variables dans les chaînes de caractères,
    # on utilise les “f-strings”
    return f"Vous avez demandé le produit {id}"

### Gabarit simples

Maintenant nous aimerions pouvoir afficher de vraies pages HTML et non de simples messages.
Pour cela, nous pouvons utiliser les ressources du moteur de rendu **Jinja2** qui est intégré dans Flask avec la fonction `render_template`.

Pour cela, il suffit d'un fichier HTML, qui sera stocké dans un sous-dossier `templates`.
Admettons que nous vi-oulions afficher une page d'accueil qui s'appelle `home.html` :

In [ ]:
from flask import render_template

@app.route('/')
def home():
    return render_template('home.html')

Naturellement, des pages statiques ne sont pas très utiles dans une application.
Nous aimerions pouvoir passer des données au gabarit.
Dans ce cas, il suffit de les ajouter comme arguments de la fontion `render_template`.

In [ ]:
products = {
    'iphone': {
        'name': 'iPhone 5S',
        'category': 'Phones',
        'price': 699,
    },
    'galaxy': {
        'name': 'Samsung Galaxy 5',
        'category': 'Phones',
        'price': 649,
    },
    'ipad-air': {
        'name': 'iPad Air',
        'category': 'Tablets',
        'price': 649,
    },
    'ipad-mini': {
        'name': 'iPad Mini',
        'category': 'Tablets',
        'price': 549
    }
}

@app.route('/product/<id>')
def home(id):
    return render_template('product.html', id=id, product=products[id])


Dans le fichier HTML correpondant, les éléments variables seront entourés de doubles accolades : `{{ id }}`.
En réalité, les doubles accolades peuvent contenir du code Python arbitraire, que Jinja se chargera d'exécuter.

```
<div class="top-pad">
    <h1>{{ product['name'] }}
      <small>{{ product['category'] }}</small>
    </h1>
    <h3>{{ product['price'] }} euros</h3>
</div>
```

**Jinja** offre également deux fonctionnalités très importantes :
- une notion d'héritage, qui permet de mutualiser certaines parties du code HTML dans des blocs
- des sructure de contrôles pour les conditionnelles et les boucles

**Flask**, quant à lui, se chargera de traiter les erreurs (comme les routes invalides, par exemple).

#### Héritage de gabarit

Si nous voulons mutualiser une artie du code HTML, pour créer une mise en page générique, par exemple, nous devons juste écrire un fichier contenant des blocs, qui sont des instructions Jinja. Ces dernières sont indiquées par des accolade-pourcent : `{% instruction %}`.
```
<html>
    <head>
        <title>{{ title }}</title>
    </head>
    <body>
        {% block contenu %}{% endblock %}
    </body>
</html>
```
Ceci est notre gabarit “parent”. Appelons-le `layout.html`.
On remarque que le bloc `contenu` est vide. C'est juste un déclaration qui sera remplie par la suite.

Pour qu'une page adopte le gabarit, il faut le lui indiquer avec l'instruction `{% extends %}`.
Il ne restera alors quà définir le contenu des blocs. Par exemple, pour reprendre la page produit :

```
{% extends 'layout.html' %}

{% block contenu %}
<div class="top-pad">
    <h1>{{ product['name'] }}
      <small>{{ product['category'] }}</small>
    </h1>
    <h3>{{ product['price'] }} euros</h3>
</div>
{% endblock %}
```

#### Structures de contrôle

##### Boucles

Si nous voulons afficher la liste des produits (sur le modèle de l'exemple plus haut), nous pourrions écrire :
```
{% extends 'layout.html' %}

{% block contenu %}
<div>
    {% for id, product in products.items() %}
     <div class="product">
        <h2>
          <a href="{{ app.url_for('product', key=id) }}">{{product['name']}}</a>
          <small>{{ product['price']}} euros</small>
        </h2>
     </div>
    {% endfor %}
</div>
{% endblock %}
```

Notez au passage l'emploi de la fonc `url_for`, qui permet de générer l'URL correpondant à une route :
- le premier argument est le nom du contrôleur associé à la route
- les arguments suivant sont les variables de la route

Dans cet exemple, nous pourrions imaginer une route :

@app.route('p

##### Conditionnelles

